**Accessing List of All Subsectors**

In [5]:
import os
import pandas as pd
from dotenv import load_dotenv
import requests


load_dotenv()
SECTOR_API_KEY = os.environ.get("SECTOR_API_KEY")

headers = {
    "Authorization": SECTOR_API_KEY
}
response = requests.get("https://api.sectors.app/v1/subsectors/", headers = headers)

if response.status_code == 200:
    data_all_subsectors = response.json()
else:
    # Handle error
    print(response.status_code)

pd.DataFrame(data_all_subsectors).head()


,sector,subsector
0,infrastructures,telecommunication
1,energy,oil-gas-coal
2,financials,financing-service
3,financials,investment-service
4,consumer-cyclicals,apparel-luxury-goods


### **Data Preparation** <p>
we're going to use subsector column, then we'll drop the sector data one.

In [6]:
# transforming data to pandas dataframe
df = pd.DataFrame(data_all_subsectors)
df = df.drop(["sector"], axis=1)
df.head()


,subsector
0,telecommunication
1,oil-gas-coal
2,financing-service
3,investment-service
4,apparel-luxury-goods


standardise the data

In [7]:
df["subsector"] = df["subsector"].str.replace("-"," ")
df["subsector"] = df["subsector"].str.title()
df.head()


,subsector
0,Telecommunication
1,Oil Gas Coal
2,Financing Service
3,Investment Service
4,Apparel Luxury Goods


### **Visualisation**

Collecting data

In [10]:
result_data = []
for data in data_all_subsectors:
    subsector = data["subsector"]
    headers = {
    "Authorization": SECTOR_API_KEY
    }

    response = requests.get(f"https://api.sectors.app/v1/companies/?sub_sector={subsector}", headers = headers)

    if response.status_code == 200:
        list_companies_by_subsectors = response.json()
        for company in list_companies_by_subsectors:
            result_data.append([subsector, company["symbol"]])
    else:
        # Handle error
        print("Error Status :",response.status_code)

print(result_data)


Error Status : 429
Error Status : 429
Error Status : 429
Error Status : 429
Error Status : 429
Error Status : 429
Error Status : 429
Error Status : 429
Error Status : 429
Error Status : 429
Error Status : 429
Error Status : 429
Error Status : 429
Error Status : 429
[['telecommunication', 'BTEL.JK'], ['telecommunication', 'EXCL.JK'], ['telecommunication', 'FREN.JK'], ['telecommunication', 'GHON.JK'], ['telecommunication', 'ISAT.JK'], ['telecommunication', 'LINK.JK'], ['telecommunication', 'MTEL.JK'], ['telecommunication', 'TBIG.JK'], ['telecommunication', 'TLKM.JK'], ['telecommunication', 'TOWR.JK'], ['telecommunication', 'KETR.JK'], ['telecommunication', 'MORA.JK'], ['telecommunication', 'INET.JK'], ['telecommunication', 'BALI.JK'], ['telecommunication', 'SUPR.JK'], ['telecommunication', 'OASA.JK'], ['telecommunication', 'CENT.JK'], ['telecommunication', 'KBLV.JK'], ['telecommunication', 'DATA.JK'], ['telecommunication', 'LCKM.JK'], ['telecommunication', 'JAST.JK'], ['telecommunication

In [11]:
df = pd.DataFrame(result_data, columns=["subsectors", "company"])
df["subsectors"] = df["subsectors"].str.replace("-"," ")
df["subsectors"] = df["subsectors"].str.title()
df_grouped = df.groupby('subsectors').size().reset_index(name="total_companies")


In [13]:
df_grouped.head()

,subsectors,total_companies
0,Alternative Energy,2
1,Apparel Luxury Goods,24
2,Automobiles Components,18
3,Banks,47
4,Consumer Services,52


In [14]:
import altair as alt

chart = alt.Chart(df_grouped).mark_bar().encode(
    x='total_companies',
    y=alt.Y('subsectors', sort='-x')
)

chart.display()


alt.Chart(...)

In [15]:
df_grouped = df_grouped.sort_values(by="total_companies", ascending=False)
top_five_subsectors = df_grouped.head(5)

In [16]:
chart = alt.Chart(top_five_subsectors).mark_bar().encode(
    y=alt.Y("subsectors", sort="-x", title="Subsectors"),
    x=alt.X("total_companies", title="Total Companies"),
    color=alt.Color('subsectors:N', legend=None)  # Optional: remove the legend if it's redundant
).properties(
    title='Number of Companies by Subsector',
    width=600,
    height=300
)

# Configure overall visual theme
chart = chart.configure_title(fontSize=20, anchor='start', color='black').configure_axis(
    labelFontSize=12,
    titleFontSize=14
)

chart.display()

alt.Chart(...)